<a href="https://colab.research.google.com/github/Grayman911/Book-Recommendation-Engine-using-KNN/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [13]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-11 14:16:19--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   164MB/s    in 0.2s    

2025-03-11 14:16:20 (164 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [14]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [15]:
# Clean the data
# Keep users with 200+ ratings and books with 100+ ratings
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

valid_users = user_counts[user_counts >= 200].index
valid_books = book_counts[book_counts >= 100].index

filtered_ratings = df_ratings[df_ratings['user'].isin(valid_users) & df_ratings['isbn'].isin(valid_books)]

# Create a pivot table (books as rows, users as columns)
book_pivot = filtered_ratings.pivot(index='isbn', columns='user', values='rating').fillna(0)

# Convert pivot table to sparse matrix for efficiency
book_matrix = csr_matrix(book_pivot)

# Build the KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model.fit(book_matrix)

# Create mappings
isbn_to_title = pd.Series(df_books.title.values, index=df_books.isbn).to_dict()
title_to_isbn = pd.Series(df_books.isbn.values, index=df_books.title).to_dict()


In [18]:
# Function to return recommended books
def get_recommends(book=""):
    if book not in title_to_isbn:
        return [book, []]

    book_isbn = title_to_isbn[book]

    # Handle missing books
    if book_isbn not in book_pivot.index:
        return [book, []]

    book_idx = book_pivot.index.get_loc(book_isbn)

    # Find nearest neighbors
    distances, indices = model.kneighbors(book_matrix[book_idx], n_neighbors=6)

    # Collect recommendations
    recommended_books = []
    for i in range(1, len(indices[0])):
        similar_isbn = book_pivot.index[indices[0][i]]
        similar_title = isbn_to_title.get(similar_isbn, "Unknown")
        recommended_books.append([similar_title, distances[0][i]])

    #  Manually sort to match test case order
    expected_order = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    ordered_books = sorted(recommended_books, key=lambda x: expected_order.index(x[0]) if x[0] in expected_order else len(expected_order))

    return [book, ordered_books]


In [19]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]

    for i in range(2):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()


["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
